In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
#from experiment_tools import *
from pix2struct import *
PROMPT_TOKEN = "<|PROMPT|>"
X_START = "<x_start>"
X_END = "<x_end>"
Y_START = "<y_start>"
Y_END = "<y_end>"

SEPARATOR_TOKENS = [PROMPT_TOKEN,
                    X_START,
                    X_END,
                    Y_START,
                    Y_END]

LINE_TOKEN =  "<line>" 
VERTICAL_BAR_TOKEN = "<vertical_bar>"
HORIZONTAL_BAR_TOKEN = "<horizontal_bar>"
SCATTER_TOKEN = "<scatter>"
DOT_TOKEN = "<dot>"

CHART_TYPE_TOKENS = [LINE_TOKEN,
                    VERTICAL_BAR_TOKEN,
                    HORIZONTAL_BAR_TOKEN,
                    SCATTER_TOKEN,
                    DOT_TOKEN]

new_tokens = SEPARATOR_TOKENS + CHART_TYPE_TOKENS

In [2]:
data_config = DataConfig(path = 'data')
dt = DataTools(data_config)

In [3]:
experiment = ModelExperiment(experiment_name='pix2struct_base', 
                             data_config = data_config,
                             )

In [4]:
dataset = dt.build_dataset(stage='dev', sample_size=20, train_val_split=0.8)
train_data, val_data = dataset['train'], dataset['val']

In [5]:
train_data[0]

{'img_path': PosixPath('data/train/images/0000ae6cbdb1.jpg'),
 'annotations': '<|PROMPT|><vertical_bar><x_start>Malawi;Malaysia;Maldives;Mali;Malta;Martinique;Mauritania;Mauritius;Mayotte;Melanesia<x_end><y_start>837799.95;960480.61;750170.90;697593.48;574912.82;452232.15;653778.96;539861.20;417180.54;574912.82<y_end>'}

In [6]:
lora_config = dict({'r':16,
                   'lora_alpha':16,
                   'target_modules':["query", "value"],
                     'lora_dropout':0.1,
                        'bias':"none"
                        })
generator = CaptionGenerator(device='cpu',
                             lora_config=None)#lora_config)

In [7]:
#generator.load_model(use_peft=True,  additional_tokens=new_tokens)
generator.load_model(use_peft=False, additional_tokens=new_tokens)

/Users/santiagoct/Desktop/_benetech_/_ben_t_comp_/envai/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
experiment.train_model(generator = generator,
                       train_dataset = train_data,
                       epochs = 1,
                       batch_size = 2,
                       NUM_ACCUMULATION_STEPS=8)

Epoch: 0
Loss: 4.464968681335449
Loss: 5.76653528213501
Loss: 6.093614101409912
Loss: 5.355437755584717
Loss: 5.803844928741455
Loss: 4.6596856117248535


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 experiment.train_model(generator = generator,                                                │
│   2 │   │   │   │   │      train_dataset = train_data,                                           │
│   3 │   │   │   │   │      epochs = 1,                                                           │
│   4 │   │   │   │   │      batch_size = 2,                                                       │
│                                                                                                  │
│ /Users/santiagoct/Desktop/_benetech_/_ben_t_comp_/pix2struct.py:485 in train_model               │
│                                                                                                  │
│   482 │   │   │   │                                                                              │
│   483 │   │   │   │   print("Loss:", loss.item())                                                │
│   484 │   │   │   │                                                                              │
│ ❱ 485 │   │   │   │   loss.backward()                                                            │
│   486 │   │   │   │                                                                              │
│   487 │   │   │   │   if ((idx + 1) % NUM_ACCUMULATION_STEPS == 0) or (idx + 1 == len(train_da   │
│   488 │   │   │   │   │   generator.optimizer.zero_grad()                                        │
│                                                                                                  │
│ /Users/santiagoct/Desktop/_benetech_/_ben_t_comp_/envai/lib/python3.11/site-packages/torch/_tens │
│ or.py:487 in backward                                                                            │
│                                                                                                  │
│    484 │   │   │   │   create_graph=create_graph,                                                │
│    485 │   │   │   │   inputs=inputs,                                                            │
│    486 │   │   │   )                                                                             │
│ ❱  487 │   │   torch.autograd.backward(                                                          │
│    488 │   │   │   self, gradient, retain_graph, create_graph, inputs=inputs                     │
│    489 │   │   )                                                                                 │
│    490                                                                                           │
│                                                                                                  │
│ /Users/santiagoct/Desktop/_benetech_/_ben_t_comp_/envai/lib/python3.11/site-packages/torch/autog │
│ rad/__init__.py:200 in backward                                                                  │
│                                                                                                  │
│   197 │   # The reason we repeat same the comment below is that                                  │
│   198 │   # some Python versions print out the first line of a multi-line function               │
│   199 │   # calls in the traceback and some print out the last line                              │
│ ❱ 200 │   Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the bac   │
│   201 │   │   tensors, grad_tensors_, retain_graph, create_graph, inputs,                        │
│   202 │   │   allow_unreachable=True, accumulate_grad=True)  # Calls into the C++ engine to ru   │
│   203                                                                                            │
╰────────────────────────────────────────────────────────────

In [44]:
#generator.model.print_trainable_parameters()

In [10]:
train_data[0]

{'img_path': PosixPath('data/train/images/0000ae6cbdb1.jpg'),
 'annotations': '<|PROMPT|><vertical_bar><x_start>Malawi;Malaysia;Maldives;Mali;Malta;Martinique;Mauritania;Mauritius;Mayotte;Melanesia<x_end><y_start>837799.95;960480.61;750170.90;697593.48;574912.82;452232.15;653778.96;539861.20;417180.54;574912.82<y_end>'}